In [ ]:
# 01 
from PyQt5.QtWidgets import *
import sys
import winsound

class BeepSound(QMainWindow):
    def __init__(self):
        # QMainWindow 초기화 (부모, 플래그 전달)
        super().__init__()
        
        # [1] 윈도우 기본 설정
        self.setWindowTitle('삑 소리 내기')
        self.setGeometry(200, 200, 500, 100)
        
        # [2] 버튼 3개 생성
        soundButton = QPushButton('소리', self)
        quitButton = QPushButton('나가기', self)
        
        # [3] 안내 라벨
        self.label = QLabel('환영합니다!', self)
        
        # [4] 버튼 위치 배치
        soundButton.setGeometry(10, 10, 100, 30)
        quitButton.setGeometry(210, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 70)
        
        # [5] 버튼 클릭 연결 (Signal-Slot 연결)
        soundButton.clicked.connect(self.soundFunction)
        quitButton.clicked.connect(self.quitFunction)
        
    # [6] 짧은 삑 소리 함수
    def soundFunction(self):
        winsound.PlaySound("SystemQuestion", winsound.SND_ALIAS)


    # [8] 종료 함수
    def quitFunction(self):
        self.close()

# [9] PyQt5 애플리케이션 실행 루프
app = QApplication(sys.argv)
win = BeepSound()
win.show()
app.exec_()

0

In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QTimer
import sys
import cv2 as cv
import numpy as np

class VideoCaptureApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('QTimer로 비디오 프레임 수집 개선 버전')
        self.setGeometry(200, 200, 800, 100)

        # 버튼들
        self.videoButton = QPushButton('비디오 켜기', self)
        self.captureButton = QPushButton('프레임 잡기', self)
        self.multiCaptureButton = QPushButton('여러 프레임 잡기', self)
        self.stackShowButton = QPushButton('프레임 이어보기', self)
        self.quitButton = QPushButton('나가기', self)

        # 버튼 배치
        self.videoButton.setGeometry(10, 10, 120, 30)
        self.captureButton.setGeometry(140, 10, 120, 30)
        self.multiCaptureButton.setGeometry(270, 10, 120, 30)
        self.stackShowButton.setGeometry(400, 10, 150, 30)
        self.quitButton.setGeometry(560, 10, 100, 30)

        # 버튼 연결
        self.videoButton.clicked.connect(self.startVideo)
        self.captureButton.clicked.connect(self.captureFrame)
        self.multiCaptureButton.clicked.connect(self.startMultiCapture)
        self.stackShowButton.clicked.connect(self.showStack)
        self.quitButton.clicked.connect(self.quitApp)

        # 초기 비활성화
        self.captureButton.setEnabled(False)
        self.multiCaptureButton.setEnabled(False)
        self.stackShowButton.setEnabled(False)

        # 내부 변수들
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.updateFrame)

        self.frame = None
        self.capturedFrame = None
        self.frames = []
        self.multiCaptureMode = False

    def startVideo(self):
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():
            print('카메라 연결 실패')
            self.close()
            return

        self.timer.start(30)  # QTimer 시작
        print("비디오 스트림 시작")

        # 버튼 상태 변경
        self.captureButton.setEnabled(True)
        self.multiCaptureButton.setEnabled(True)
        self.stackShowButton.setEnabled(True)

    def updateFrame(self):
        ret, self.frame = self.cap.read()
        if not ret:
            return

        cv.imshow('Video Stream', self.frame)

        key = cv.waitKey(1)
        if self.multiCaptureMode and key == ord('c'):
            self.frames.append(self.frame.copy())
            print(f"프레임 {len(self.frames)} 저장됨")

    def captureFrame(self):
        if self.frame is not None:
            self.capturedFrame = self.frame.copy()
            cv.imshow('Captured Frame', self.capturedFrame)
            print("단일 프레임 저장 완료!")
        else:
            print("캡처할 프레임이 없습니다!")

    def startMultiCapture(self):
        if self.cap is None or not self.cap.isOpened():
            print("비디오를 먼저 켜세요!")
            return

        self.frames = []
        self.multiCaptureMode = True
        print("여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료")

    def showStack(self):
        if len(self.frames) == 0:
            print("저장된 프레임이 없습니다! 먼저 여러 프레임을 잡으세요.")
            return

        stack = cv.resize(self.frames[0], dsize=(0, 0), fx=0.3, fy=0.3)
        for i in range(1, len(self.frames)):
            resized = cv.resize(self.frames[i], dsize=(0, 0), fx=0.3, fy=0.3)
            stack = np.hstack((stack, resized))

        cv.imshow('Stacked Frames', stack)
        print(f"총 {len(self.frames)}개의 프레임 이어서 표시됨")

    def quitApp(self):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
        cv.destroyAllWindows()
        self.close()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    win = VideoCaptureApp()
    win.show()
    app.exec_()



비디오 스트림 시작
여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료
저장된 프레임이 없습니다! 먼저 여러 프레임을 잡으세요.
단일 프레임 저장 완료!
단일 프레임 저장 완료!
단일 프레임 저장 완료!
저장된 프레임이 없습니다! 먼저 여러 프레임을 잡으세요.
여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료
단일 프레임 저장 완료!
여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료
프레임 1 저장됨
프레임 2 저장됨
프레임 3 저장됨
총 3개의 프레임 이어서 표시됨
